# Building a CV Model
## Importing Data from Roboflow

In [ ]:
import os
from roboflow import Roboflow
import yaml


rf = Roboflow(api_key='CBfyNcr7kbpR77E7GwFx')

project = rf.workspace('fbridgeman').project('grey-to-green-reduced')

# Get working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# ask if the user wants to deploy the model with yes or y
user_input = input("Do you want to download the dataset inside this current working directory? (yes/y): ").strip().lower()
if user_input not in ['yes', 'y']:
    print("Deployment aborted.")
    exit()
else:
    print("Proceeding with download...")

user_input2 = input("What do you want the data directory to be called? ").strip().lower()
version_name = input("What version number do you want to deploy? ").strip().lower()


data_location = os.path.join(current_dir, user_input2)
# data_location = os.path.join(current_dir, 'data')
# print(f"Data location: {data_location}")

# Ensure the data directory exists
if not os.path.exists(data_location):
    os.makedirs(data_location)
    print(f"Created data directory at: {data_location}")
else:
    print(f"Data directory already exists at: {data_location}")

# Download the dataset
dataset = project.version(version_name).download(model_format='yolov8', location = data_location)# , location=data_location

# with open(f"{dataset.location}/data.yaml", 'r') as f:
#     dataset_yaml = yaml.safe_load(f)
# dataset_yaml["train"] = "../train/images"
# dataset_yaml["val"] = "../valid/images"
# dataset_yaml["test"] = "../test/images"
# with open(f"{dataset.location}/data.yaml", 'w') as f:
#     yaml.dump(dataset_yaml, f)


# Check the location of the data directory
print(f"Dataset downloaded to: {dataset.location}")

# Check the contents of the data directory
data_contents = os.listdir(dataset.location)
print(f"Data directory contents: {data_contents}")




# Train a YOLO 11 Model

In [ ]:
# Yale HPC
# Trial 2 / Generalized Code
# Updated to take in user settings 12 NOV 2024


from ultralytics import YOLO
import os

# Update ultralytics settings
# PROMPT 1
dataset_input = input("What dataset version do you want to be called? ").strip().lower()
# PROMPT 2
project_input = input("What project version do you want to be called? ").strip().lower()

# List of available models
models = {
    "1": "yolov8n.pt",
    "2": "yolov8m.pt",
    "3": "yolov8l.pt",
    "4": "yolov8x.pt",
    "5": "yolo11n.pt",
    "6": "yolo11m.pt",
    "7": "yolo11l.pt",
    "8": "yolo11x.pt",
}

# Display the available models
print("Available models:")
for key, value in models.items():
    print(f"{key}: {value}")

# PROMPT 3
# Prompt the user to choose a model
model_choice = input("Enter the number of the model you want to load: ").strip()

# Validate the choice and load the model
if model_choice in models:
    chosen_model = models[model_choice]
    model = YOLO(chosen_model)
    print(f"Loaded model: {chosen_model}")
else:
    model = YOLO("yolo11x.pt")
    print("Invalid choice. Defaulted to yolo11x.")

# PROMPT 4
gpu_input = input("How many GPUs have you allocated? ").strip().lower()

# PROMPT 5
run_input = input("What do you want this run do you want to be called? ").strip().lower()

# data_location = os.path.join("/gpfs/gibbs/project/miranda/fwb7/yolov8/", dataset_input, "data.yaml")
data_location = os.path.join("/vast/palmer/scratch/miranda/fwb7/yolov8", dataset_input, "data.yaml")
project_location = os.path.join("/gpfs/gibbs/project/miranda/fwb7/yolov8/", project_input)

if gpu_input == "1":
    gpu_num = 0
elif gpu_input == "2":
    gpu_num = [0,1]
elif gpu_input == "3":
    gpu_num = [0,1,2]
elif gpu_input == "4":
    gpu_num = [0,1,2,3]
else:
    gpu_num = None

# Ensure the project directory exists
if not os.path.exists(project_location):
    os.makedirs(project_location)
    print(f"Created project directory at: {project_location}")
else:
    print(f"Project directory already exists at: {project_location}")

print(data_location)
print(project_location)

# Ask user if they want to continue:
# response = input("Do you want to continue? (yes/no): ").strip().lower()
# if response not in ['yes', 'y']:
#     print("Exiting the program.")
#     exit()
# else:
#     print("Continuing...")

# project.version(DATASET_VERSION).deploy(model_type=”yolov8”, model_path=f”{HOME}/runs/detect/train/”)
# Train the model with MPS
model.train(name = run_input,
              data = data_location,
              project = project_location,
              epochs = 50,
              batch = 16,
              imgsz = 416,
              device = gpu_num,
              verbose = True,
              time = 2,
              plots = True)

# Finetune model
model.tune()

## Validate Model
Validating 

In [ ]:

from ultralytics import YOLO
import ultralytics as ultra
# from IPython.display import display, Image
import os
import yaml
import json
import pandas as pd

# Update ultralytics settings
data_dir = "/gpfs/gibbs/project/miranda/fwb7/yolov8/v2/data.yaml"
project_dir = "/gpfs/gibbs/project/miranda/fwb7/yolov8/beta2"
gpu_num = [0,1,2,3]
selected_model_name = "v2_data_yolo11x2"
args = "args.yaml"
model_dir = os.path.join(project_dir, selected_model_name)
model_args_dir = os.path.join(model_dir, args)
weights = 'weights/best.pt'
weights_dir = os.path.join(model_dir, weights)
print(data_dir)
print(project_dir)
print(selected_model_name)

# PROMPT 1
#gpu_input = input("How many GPUs have you allocated? ").strip().lower()

# Print ultralytics settings
# print(ultra.settings())

# Ask user if they want to continue:
# response = input("Do you want to continue? (yes/no): ").strip().lower()
# if response not in ['yes', 'y']:
#     print("Exiting the program.")
#     exit()
# else:
#     print("Continuing...")

# Provide a list of models that are directories within "/gpfs/gibbs/project/miranda/fwb7/yolov8/beta1"
# model_directory = "/gpfs/gibbs/project/miranda/fwb7/yolov8/beta1"
# models = [d for d in os.listdir(model_directory) if os.path.isdir(os.path.join(model_directory, d))]

# print("Available models:")
# for model in models:
#     print(model)

# Ask user which model they want to load
#selected_model_name = input(f"Which model do you want to load? ({'Choice: '.join(models)}): ").strip().lower()
# selected_model_location = os.path.join(model_directory, selected_model_name)
# selected_model_args = os.path.join(selected_model_location, 'args.yaml')

# print("Selected model location:", selected_model_location)

# Read original params from the args.yaml file
with open(model_args_dir, 'r') as file:
    args = yaml.load(file, Loader=yaml.FullLoader)
    # Original dataset location
    original_data_location = args['data']
    # Original batch size
    og_batch_size = args['batch']
    # Original image size
    og_image_size = args['imgsz']
    # Original epochs
    og_epochs = args['epochs']

# model = YOLO()  # load train2 model
model = YOLO(weights_dir)
print("Selected model name", model.model_name)
print("Validating dataset saved in:", original_data_location)

# validation name
validation_name = f"{selected_model_name}_VAL"
validation_dir = os.path.join(project_dir, validation_name)

# validate the model
print("VALIDATING THE MODEL NOW")
results = model.val(data = original_data_location,
                    name = validation_name,
                    project = project_dir,
                    device = gpu_num,
                    imgsz = og_image_size,
                    epochs = og_epochs,
                    batch = 16,
                    verbose = True,
                    plots = True,
                    save_json = True)

# print the results
print(results)
print(f"mAP50: {results.box.map50}")  # map50 [1]
print(f"mAP50-95: {results.box.map}")  # map50-95 [1]
# print(f"mAP50 for each category: {results.box.map50s}")  # list of mAP50 for each category
# print(f"mAP50-95 for each category: {results.box.maps}")  # list of mAP50-95 for each category

# # convert saved results from JSON file to CSV
# # Assuming your JSON results are saved as 'results.json'
# results_json = f"{project_dir}/results/{validation_name}/results.json"

# # read the JSON file
# with open(results_json, 'r') as file:
#     data = json.load(file)

# # Convert JSON to DataFrame then save as CSV
# df = pd.DataFrame(data)

# # Print df
# print("Validation results:")
# print(df)

# # Ask if user wants to save as csv
# #response = input("Do you want to save the results as CSV? (yes/no): ").strip().lower()
# response = "yes"
# if response not in ['yes', 'y']:
#     print("Exiting the program.")
#     exit()
# else:
#     # Save as CSV
#     df.to_csv(os.path.join(save_dir, 'validation_results.csv'), index=False)
#     print(f"Validation results saved as 'validation_results.csv' in {validation_dir}")


# Finetune Hyperparameters

In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("/gpfs/gibbs/project/miranda/fwb7/yolov8/beta2/v2_data_yolo11x2/weights/best.pt")
data = "/gpfs/gibbs/project/miranda/fwb7/yolov8/v2/data.yaml"
# Tune hyperparameters using Ray_Tune
result_grid = model.tune(data=data, use_ray = True, device = [0,1,2,3]
                         

## Calculating results

In [ ]:
from ultralytics import YOLO
import ultralytics as ultra
# from IPython.display import display, Image
import os
import yaml
import json
import pandas as pd